### Real-Time Traffic Sign Detection Using Webcam

This notebook demonstrates real-time traffic sign recognition using a webcam.
A trained CNN model is loaded and used to classify traffic signs from live video frames.
Predicted class labels and confidence scores are displayed on the video stream.



In [31]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input



### Load Trained CNN Model

This section loads the previously trained and saved CNN model.
The model will be used to predict traffic sign classes from webcam frames.


In [32]:
MODEL_PATH = "../models/traffic_sign_mobilenet.keras"
model = load_model(MODEL_PATH)

print("Model loaded successfully")


Model loaded successfully


### Load Class Labels

This section loads the class labels based on the training dataset folders.
The order of these labels must match the order used during training.


In [33]:
TRAIN_DIR = "../data/Train"
class_labels = sorted(os.listdir(TRAIN_DIR))

print("Classes:", class_labels)
print("Number of classes:", len(class_labels))


Classes: ['ALL_MOTOR_VEHICLE_PROHIBITED', 'AXLE_LOAD_LIMIT', 'BARRIER_AHEAD', 'BULLOCK_AND_HANDCART_PROHIBITED', 'BULLOCK_PROHIBITED', 'CATTLE', 'COMPULSARY_AHEAD', 'COMPULSARY_AHEAD_OR_TURN_LEFT', 'COMPULSARY_AHEAD_OR_TURN_RIGHT', 'COMPULSARY_CYCLE_TRACK', 'COMPULSARY_KEEP_LEFT', 'COMPULSARY_KEEP_RIGHT', 'COMPULSARY_MINIMUM_SPEED', 'COMPULSARY_SOUND_HORN', 'COMPULSARY_TURN_LEFT', 'COMPULSARY_TURN_LEFT_AHEAD', 'COMPULSARY_TURN_RIGHT', 'COMPULSARY_TURN_RIGHT_AHEAD', 'CROSS_ROAD', 'CYCLE_CROSSING', 'CYCLE_PROHIBITED', 'DANGEROUS_DIP', 'DIRECTION', 'FALLING_ROCKS', 'FERRY', 'GAP_IN_MEDIAN', 'GIVE_WAY', 'GUARDED_LEVEL_CROSSING', 'HANDCART_PROHIBITED', 'HEIGHT_LIMIT', 'HORN_PROHIBITED', 'HUMP_OR_ROUGH_ROAD', 'LEFT_HAIR_PIN_BEND', 'LEFT_HAND_CURVE', 'LEFT_REVERSE_BEND', 'LEFT_TURN_PROHIBITED', 'LENGTH_LIMIT', 'LOAD_LIMIT', 'LOOSE_GRAVEL', 'MEN_AT_WORK', 'NARROW_BRIDGE', 'NARROW_ROAD_AHEAD', 'NO_ENTRY', 'NO_PARKING', 'NO_STOPPING_OR_STANDING', 'OVERTAKING_PROHIBITED', 'PASS_EITHER_SIDE', 'PED

### Prediction Stabilization Techniques

This section improves real-time prediction stability by applying

In [34]:
CONFIDENCE_THRESHOLD = 0.30   # Ignore weak predictions
SMOOTHING_FRAMES = 10         # Number of frames to smooth over


### Frame Preprocessing

This function preprocesses webcam frames before prediction.
Frames are resized and normalized to match the CNN input requirements.


In [38]:
IMG_SIZE = (96, 96)

def preprocess_frame(frame):
    img = cv2.resize(frame, IMG_SIZE)
    img = preprocess_input(img)
    img = np.expand_dims(img, axis=0)
    return img


In [36]:
from collections import deque

prediction_buffer = deque(maxlen=SMOOTHING_FRAMES)


### Real-Time Prediction Loop

This section captures live video from the webcam, processes each frame,
predicts the traffic sign class, and displays the result with confidence.


In [39]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Unable to access webcam")
    exit()

current_label = "Place sign inside box"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    # Define ROI (center box)
    box_size = 300
    x1 = w//2 - box_size//2
    y1 = h//2 - box_size//2
    x2 = w//2 + box_size//2
    y2 = h//2 + box_size//2

    roi = frame[y1:y2, x1:x2]

    processed = preprocess_frame(roi)
    prediction = model.predict(processed, verbose=0)

    class_id = np.argmax(prediction)
    confidence = np.max(prediction)

    if confidence > CONFIDENCE_THRESHOLD:
        label = class_labels[class_id]
        text = f"{label} ({confidence*100:.1f}%)"
    else:
        text = "Low confidence"

    # Draw ROI box
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
    cv2.putText(frame, text, (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow("Traffic Sign Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
